In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

In [37]:
## https://finance.naver.com/sise/sise_market_sum.naver

# 번호 : #contentarea > div.box_type_l > table.type_2 > tbody > tr:nth-child(2) > td.no
# 종목명 : #contentarea > div.box_type_l > table.type_2 > tbody > tr:nth-child(2) > td:nth-child(2) > a
# contentarea > div.box_type_l > table.type_2 > tbody > tr:nth-child(3) > td:nth-child(2) > a
# 현재가 : #contentarea > div.box_type_l > table.type_2 > tbody > tr:nth-child(2) > td:nth-child(3)
# 전일비 : #contentarea > div.box_type_l > table.type_2 > tbody > tr:nth-child(2) > td:nth-child(4) > span
# 등락률 : #contentarea > div.box_type_l > table.type_2 > tbody > tr:nth-child(2) > td:nth-child(5) > span
# 액면가 : #contentarea > div.box_type_l > table.type_2 > tbody > tr:nth-child(2) > td:nth-child(6)
url = "https://finance.naver.com/sise/sise_market_sum.naver"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
}
# 페이지 호출
res = requests.get(url, headers=headers)
soup = bs(res.content.decode("euc-kr", "replace"), "lxml")

col = soup.select("#contentarea > div.box_type_l > table.type_2 > thead > tr > th")
번호 = soup.select("#contentarea > div.box_type_l > table.type_2 > tbody > tr > td.no")
종목명 = soup.select(
    "#contentarea > div.box_type_l > table.type_2 > tbody > tr > td:nth-child(2)"
)
현재가 = soup.select(
    "#contentarea > div.box_type_l > table.type_2 > tbody > tr > td:nth-child(3)"
)
전일비 = soup.select(
    "#contentarea > div.box_type_l > table.type_2 > tbody > tr > td:nth-child(4)"
)
등락률 = soup.select(
    "#contentarea > div.box_type_l > table.type_2 > tbody > tr > td:nth-child(5)"
)
액면가 = soup.select(
    "#contentarea > div.box_type_l > table.type_2 > tbody > tr > td:nth-child(6)"
)
번호List = []
종목명List = []
현재가List = []
전일비List = []
등락률List = []
액면가List = []

for i in range(len(번호)):
    번호List.append(번호[i].text)
    종목명List.append(종목명[i].text)
    현재가List.append(현재가[i].text)
    전일비List.append(전일비[i].text.strip())
    등락률List.append(등락률[i].text.strip())
    액면가List.append(액면가[i].text)

dic = {
    col[0].text: 번호List,
    col[1].text: 종목명List,
    col[2].text: 현재가List,
    col[3].text: 전일비List,
    col[4].text: 등락률List,
    col[5].text: 액면가List,
}

증권 = pd.DataFrame(dic)

증권.set_index(col[1].text, inplace=True)

증권.to_csv('네이버증권오늘의시세.csv', encoding='utf-8')